# αMCTS

**Game trees** are a way to represent game states and actions. 

Each node in the tree represents a game state, and each edge represents a possible action that can be taken from that state.

In [ ]:
import math


def draw_tic_tac_toe(board, size=300, padding=20):
    """
    Draw a tic-tac-toe board as SVG
    """
    cell_size = (size - padding * 2) / 3
    svg = f'<svg width="{size}" height="{size}" style="display:block;">'
    # Background
    svg += f'<rect x="{padding}" y="{padding}" width="{size - padding * 2}" height="{size - padding * 2}" fill="#fafafa00" stroke-width="0" rx="4"></rect>'
    # Grid lines
    for i in range(1, 3):
        # Vertical
        x = padding + i * cell_size
        svg += f'<line x1="{x}" y1="{padding}" x2="{x}" y2="{size - padding}" stroke="#666" stroke-width="3"></line>'
        # Horizontal
        y = padding + i * cell_size
        svg += f'<line x1="{padding}" y1="{y}" x2="{size - padding}" y2="{y}" stroke="#666" stroke-width="3"></line>'
    # Marks
    for i in range(9):
        row = i // 3
        col = i % 3
        cx = padding + col * cell_size + cell_size / 2
        cy = padding + row * cell_size + cell_size / 2
        val = board[i] if i < len(board) else 0
        if val == 1:
            size_x = cell_size * 0.5
            svg += f'<line x1="{cx - size_x / 2}" y1="{cy - size_x / 2}" x2="{cx + size_x / 2}" y2="{cy + size_x / 2}" stroke="#0969da" stroke-width="3" stroke-linecap="round"></line>'
            svg += f'<line x1="{cx + size_x / 2}" y1="{cy - size_x / 2}" x2="{cx - size_x / 2}" y2="{cy + size_x / 2}" stroke="#0969da" stroke-width="3" stroke-linecap="round"></line>'
        elif val == -1:
            svg += f'<circle cx="{cx}" cy="{cy}" r="{cell_size * 0.25}" fill="none" stroke="#DA7A09" stroke-width="3"></circle>'
    svg += "</svg>"
    return svg


def draw_game_tree(root_node, board_size=120, board_padding=12):
    """
    Draw a game tree with root state and its children.
    Supports recursive tree structure.

    Args:
        root_node: Dictionary with 'state', 'children' (list of dicts), and optional 'action'
        board_size: Size of each board
        board_padding: Padding for each board

    Returns:
        HTML string with the complete game tree visualization
    """
    horizontal_spacing = 160
    vertical_spacing = 250

    # Calculate tree dimensions and positions recursively
    def get_subtree_width(node):
        if not node.get("children"):
            return horizontal_spacing
        return sum(get_subtree_width(child) for child in node["children"])

    positions = {}

    def assign_positions(node, x, y):
        positions[id(node)] = (x, y)
        if not node.get("children"):
            return

        children = node["children"]
        total_width = sum(get_subtree_width(child) for child in children)
        start_x = x - total_width / 2

        current_x = start_x
        for child in children:
            child_width = get_subtree_width(child)
            assign_positions(child, current_x + child_width / 2, y + vertical_spacing)
            current_x += child_width

    tree_width = get_subtree_width(root_node) + 20

    def get_depth(node):
        if not node.get("children"):
            return 1
        return 1 + max(get_depth(child) for child in node["children"])

    tree_height = (get_depth(root_node) * vertical_spacing + board_size) - 200

    root_x = tree_width / 2
    root_y = 40
    assign_positions(root_node, root_x, root_y)

    html = f'<div style="width:{tree_width}px;height:{tree_height}px;position:relative;margin:20px auto;">'
    svg_connections = f'<svg width="{tree_width}" height="{tree_height}" style="position:absolute;top:0;left:0;pointer-events:none;">'
    svg_connections += '<defs><marker id="arrow" viewBox="0 0 10 10" refX="6" refY="5" markerWidth="6" markerHeight="6" orient="auto"><path d="M 0 0 L 10 5 L 0 10 z" fill="#999"></path></marker></defs>'

    nodes_html = ""

    def draw_node_recursive(node, depth=0):
        nonlocal svg_connections, nodes_html
        x, y = positions[id(node)]

        # Draw node
        delay = depth * 0.5
        node_svg = draw_tic_tac_toe(
            node["state"], size=board_size, padding=board_padding
        )
        nodes_html += f'<div class="" style="position:absolute;left:{x - board_size / 2}px;top:{y}px;">{node_svg}</div>'

        if node.get("children"):
            for child in node["children"]:
                cx, cy = positions[id(child)]

                # Edge
                start_y = y + board_size
                end_y = cy

                # Draw line
                svg_connections += f'<line x1="{x}" y1="{start_y}" x2="{cx}" y2="{end_y}" stroke="#999" stroke-width="2" marker-end="url(#arrow)"  ></line>'

                # Label
                if "action" in child:
                    # Midpoint for label
                    label_x = x + (cx - x) * 0.4
                    label_y = start_y + (end_y - start_y) * 0.4

                    text = f"Action: {child['action']}"
                    text_width = len(text) * 6.5

                    # White background for readability
                    svg_connections += f'<rect x="{label_x - text_width / 2}" y="{label_y - 10}" width="{text_width}" height="20" fill="white" fill-opacity="1" />'

                    # text
                    svg_connections += f'<text x="{label_x}" y="{label_y + 5}" text-anchor="middle" fill="#999" font-size="12" font-family="Noto Sans" >{text}</text>'

                draw_node_recursive(child, depth + 1)

    draw_node_recursive(root_node)

    svg_connections += "</svg>"
    html += svg_connections
    html += nodes_html
    html += "</div>"

    return html


# Example usage
from IPython.display import HTML, display

# Root state: O's turn to play
root_state = [
    0,
    1,
    0,  # Row 1: _, X, _
    0,
    1,
    -1,  # Row 2: _, X, O
    0,
    -1,
    1,
]  # Row 3: _, O, X

# Find all possible actions (empty cells)
possible_actions = [i for i in range(9) if root_state[i] == 0]

# Create child states for each possible action
child_states = []
for action in possible_actions:
    new_state = root_state.copy()
    new_state[action] = -1  # O plays (player -1)
    child_states.append({"action": action, "state": new_state})

# Add grandchildren to action 0 (second level of the tree)
# After O plays at position 0, it's X's turn
action_0_state = child_states[0]["state"]
action_0_possible_moves = [i for i in range(9) if action_0_state[i] == 0]

grandchildren = []
for action in action_0_possible_moves:
    new_state = action_0_state.copy()
    new_state[action] = 1  # X plays (player 1)
    grandchildren.append({"action": action, "state": new_state})

# Attach grandchildren to the first child (action 0)
child_states[0]["children"] = grandchildren

# Create root node
root_node = {"state": root_state, "children": child_states}

# Generate and display the game tree
tree_html = draw_game_tree(root_node)
display(HTML(tree_html))


**Monte Carlo Tree Search (MCTS)** is a method for building game trees by simulating random games from the current state. The basic idea is to use random simulations to estimate the value of each action, and then use these estimates to guide the search for the best action.

AlphaZero uses a modified version of MCTS. The key difference is that instead of using random simulations, **αMCTS** uses a neural network to evaluate the value of each state and the probability of each action. This allows αMCTS to make more informed decisions about which actions to explore.

Three main steps in αMCTS:
1. Select
2. Expand & Evaluate
3. Backup

```mermaid
---
config:
  theme: 'base'
  themeVariables:
    primaryColor: '#ffffff00'
    primaryTextColor: '#4D5461'
    primaryBorderColor: '#096bda'
    lineColor: '#4D5461'
    secondaryColor: '#006100'
    tertiaryColor: '#4D5461'
---
stateDiagram-v2
s1: Select
s2: Expand & Evaluate
s3: Backup
[*] --> s1
s1 --> s2
s2 --> s3
s3 --> s1 : # simulations
s3 --> [*]
```

Conside the current state of the game as the root node of the tree. The αMCTS algorithm will perform a number of simulations to build the tree and estimate the value of each action.


1. **Select**: Starting from the root node, repeatedly select child nodes according to a selection policy until a leaf node is reached. The selection policy balances exploration and exploitation, using PUCT (Predictor + Upper Confidence Bound for Trees). The prior / predictor comes from the neural network's policy output.

```python
# pseudocode
def puct_value(node, child, c_puct=1.0):
    Q = child.value_sum / (1 + child.visit_count)  # Average value of the child node

    P = child.prior  # Prior probability from the neural network
    N = node.visit_count  # Total visit count of the parent node
    n = child.visit_count  # Visit count of the child node
    u = c_puct * P * (N ** 0.5) / (1 + n)
    
    return Q + u
``` 

$$\text{PUCT}(s,a) \;=\; Q(s,a) \;+\; U(s,a)$$

$$Q(s,a) \;=\; \frac{\mathrm{value\_sum}(s,a)}{\,1+\mathrm{visit\_count}(s,a)\,}$$

$$U(s,a) \;=\; c_{\mathrm{puct}}\;\cdot\; \mathrm{prior}(s,a)\;\cdot\;\frac{\sqrt{ \mathrm{visit\_count}(s,b)}}{\,1+\mathrm{visit\_count}(s,a)\,}$$

Where
- $s$ = current state, $a$ = action (child).
- $\mathrm{value\_sum}(s,a)$ = cumulative backed-up values through edge (s,a).
- $\mathrm{visit\_count}(s,a)$ = number of visits to (s,a).
- $\sum_b \mathrm{visit\_count}(s,b)$ = total visits of the parent (i.e., N_{\text{parent}}).
- $\mathrm{prior}(s,a)$ = policy prior from the network at s.
- $c_{\mathrm{puct}}>0$ = exploration constant.

We run these three steps for a number of iterations. 

1. Select a child node by using PUCT
2. Expand the node i.e. add child nodes to the selected node, and evaluate the new nodes using the neural network
3. Backup the values to the root node
 

The aim is, give a state of the game, to build a tree of possible future states by exploring possible actions. This tree can then be used to make decisions about which action to take in the current state.


Three main steps in αMCTS:
1. Selection: Starting from the root node, recursively select child nodes until a leaf node is reached. The selection is based on a balance between exploration (trying out less-visited nodes) and exploitation (choosing nodes with high value estimates).
2. Expansion: If the leaf node is not a terminal state, expand the tree by adding one or more child nodes representing possible actions from the leaf node.
3. Simulation and Backpropagation: Use the neural network to evaluate the value of the new node and the probabilities of each action. This information is then backpropagated up the tree to update the value estimates and visit counts of the nodes along the path from the root to the new node.

Let's implement a simple version of αMCTS for 2D Tic-Tac-Toe!

Lets assume we have good policy and value networks already trained for Tic-Tac-Toe.

Start with a random state 

In [1]:
print("testing")

testing


 It is O's turn to play. There are 4 possible actions to choose from this state.

In [ ]:
def draw_tic_tac_toe(board, size=300, padding=20):
    cell_size = (size - padding * 2) / 3
    svg = f'<svg width="{size}" height="{size}" style="display:block;">'
    # Background
    svg += f'<rect x="{padding}" y="{padding}" width="{size - padding * 2}" height="{size - padding * 2}" fill="#fafafa00" stroke-width="0" rx="4"></rect>'
    # Grid lines
    for i in range(1, 3):
        # Vertical
        x = padding + i * cell_size
        svg += f'<line x1="{x}" y1="{padding}" x2="{x}" y2="{size - padding}" stroke="#666" stroke-width="3"></line>'
        # Horizontal
        y = padding + i * cell_size
        svg += f'<line x1="{padding}" y1="{y}" x2="{size - padding}" y2="{y}" stroke="#666" stroke-width="3"></line>'
    # Marks
    for i in range(9):
        row = i // 3
        col = i % 3
        cx = padding + col * cell_size + cell_size / 2
        cy = padding + row * cell_size + cell_size / 2
        val = board[i] if i < len(board) else 0
        if val == 1:
            size_x = cell_size * 0.5
            svg += f'<line x1="{cx - size_x / 2}" y1="{cy - size_x / 2}" x2="{cx + size_x / 2}" y2="{cy + size_x / 2}" stroke="#0969da" stroke-width="5" stroke-linecap="round"></line>'
            svg += f'<line x1="{cx + size_x / 2}" y1="{cy - size_x / 2}" x2="{cx - size_x / 2}" y2="{cy + size_x / 2}" stroke="#0969da" stroke-width="5" stroke-linecap="round"></line>'
        elif val == -1:
            svg += f'<circle cx="{cx}" cy="{cy}" r="{cell_size * 0.25}" fill="none" stroke="#DA7A09" stroke-width="5"></circle>'
    svg += "</svg>"
    return svg


from IPython.display import HTML, display

# Root state: O's turn
root_state = [0, 1, 0, 0, 1, -1, 0, -1, 1]

# Calculate possible actions (empty cells)
possible_actions = [i for i in range(9) if root_state[i] == 0]

# Create child states
child_states = []
for action in possible_actions:
    new_state = root_state.copy()
    new_state[action] = -1  # O plays
    child_states.append({"action": action, "state": new_state})

# Layout parameters
board_size = 180
board_padding = 12
horizontal_spacing = 320
vertical_spacing = 350
tree_width = len(child_states) * horizontal_spacing
tree_height = vertical_spacing * 2 + board_size

# Build HTML
html = f'<div style="width:{tree_width}px;height:{tree_height}px;position:relative;margin:20px auto;">'

# SVG for connections
svg_connections = f'<svg width="{tree_width}" height="{tree_height}" style="position:absolute;top:0;left:0;">'

# defs for arrow
svg_connections += '<defs><marker id="arrow" viewBox="0 0 10 10" refX="6" refY="5" markerWidth="6" markerHeight="6" orient="auto"><path d="M 0 0 L 10 5 L 0 10 z" fill="#999"></path></marker></defs>'

root_x = tree_width / 2
root_y = 40

# Root board
root_svg = draw_tic_tac_toe(root_state, size=board_size, padding=board_padding)
html += f'<div style="position:absolute;left:{root_x - board_size / 2}px;top:{root_y}px;">{root_svg}</div>'

# Circle around root
svg_connections += f'<circle cx="{root_x}" cy="{root_y + board_size / 2}" r="{board_size / 2 + 10}" fill="none" stroke="#ccc" stroke-width="2"></circle>'

child_y = root_y + vertical_spacing
start_x = (tree_width - (len(child_states) - 1) * horizontal_spacing) / 2

# Draw children
for i, child in enumerate(child_states):
    child_x = start_x + i * horizontal_spacing

    # Draw curved connection path with arrow
    r = board_size / 2 + 10
    start_y = root_y + board_size / 2 + r
    end_y = child_y + board_size / 2 - r
    control_y = (start_y + end_y) / 2
    d = f"M {root_x} {start_y} Q {child_x} {control_y} {child_x} {end_y}"
    svg_connections += f'<path d="{d}" stroke="#999" stroke-width="2" fill="none" marker-end="url(#arrow)"></path>'

    # Draw child board
    child_svg = draw_tic_tac_toe(child["state"], size=board_size, padding=board_padding)
    html += f'<div style="position:absolute;left:{child_x - board_size / 2}px;top:{child_y}px;">{child_svg}</div>'

    # Add circle around child
    svg_connections += f'<circle cx="{child_x}" cy="{child_y + board_size / 2}" r="{board_size / 2 + 10}" fill="none" stroke="#ccc" stroke-width="2"></circle>'

svg_connections += "</svg>"
html += svg_connections
html += "</div>"

display(HTML(html))

In [4]:
# Nodes of tree
class Node:
    def __init__(self, state, parent=None, action=None, prior=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.children = {}
        self.visit_count = 0
        self.value_sum = 0.0
        self.prior = prior

**state**: Array of 9 elements representing the board (0 = empty, 1 = X, -1 = O)

**parent**: Reference to the parent node

**action**: Integer from 0 to 8 representing the position to place the mark

**children**: Dictionary mapping actions to child nodes

**visit_count**: Number of times the node has been visited

**value_sum**: Sum of the values of the node

**prior**: Prior probability of selecting this node

## Selection

At each node, we select the child node that maximizes the PUCT (Prior Upper Confidence applied to Trees). 

$$ PUCT = Q + U $$

$$ Q = \frac{\text{value\_sum}}{1 + \text{visit\_count}} $$

$$ U = c_{\text{puct}} \cdot \text{prior} \cdot \frac{\sqrt{\text{parent\_visit\_count}}}{1 + \text{visit\_count}} $$

Generally, Q is the average value of the node, the *exploitation* term, and U is the *exploration* term. c_puct is a constant that controls the balance between exploration and exploitation.

Prior is the probability of selecting this action, as given by the policy network.

PUCT derives from the Upper Confidence Bound (UCB) algorithm used in multi-armed bandit problems.

Let's break it down.

In [5]:
# add function to the Node class


def select_child(self, c_puct):
    best_score = -float("inf")
    best_action = None
    best_child = None

    for action, child in self.children.items():
        ucb_score = child.value_sum / (1 + child.visit_count) + c_puct * child.prior * (
            self.visit_count**0.5
        ) / (1 + child.visit_count)
        if ucb_score > best_score:
            best_score = ucb_score
            best_action = action
            best_child = child

    return best_action, best_child